In [ ]:
sbox = [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
		0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
		0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
		0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
		0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
		0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
		0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
		0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
		0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
		0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
		0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
		0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
		0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
		0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
		0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
		0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16]

Rcon = [0x00000000, 0x01000000, 0x02000000,
		0x04000000, 0x08000000, 0x10000000,
		0x20000000, 0x40000000, 0x80000000,
		0x1b000000, 0x36000000]

def keyExpansion(key):
	#prep w list to hold 44 tuples
	w = [()]*44

	#fill out first 4 words based on the key
	for i in range(4):
		w[i] = (key[4*i], key[4*i+1], key[4*i+2], key[4*i+3])

	#fill out the rest based on previews words, rotword, subword and rcon values
	for i in range(4, 44):
		#get required previous keywords
		temp = w[i-1]
		word = w[i-4]

		#if multiple of 4 use rot, sub, rcon etc
		if i % 4 == 0:
			x = RotWord(temp)
			y = SubWord(x)
			rcon = Rcon[int(i/4)]

			temp = hexor(y, hex(rcon)[2:])

		#creating strings of hex rather than tuple
		word = ''.join(word)
		temp = ''.join(temp)

		#xor the two hex values
		xord = hexor(word, temp)
		w[i] = (xord[:2], xord[2:4], xord[4:6], xord[6:8])

	return w

#takes two hex values and calculates hex1 xor hex2
def hexor(hex1, hex2):
	#convert to binary
	bin1 = hex2binary(hex1)
	bin2 = hex2binary(hex2)

	#calculate
	xord = int(bin1, 2) ^ int(bin2, 2)

	#cut prefix
	hexed = hex(xord)[2:]

	#leading 0s get cut above, if not length 8 add a leading 0
	if len(hexed) != 8:
		hexed = '0' + hexed

	return hexed

#takes a hex value and returns binary
def hex2binary(hex):
	return bin(int(str(hex), 16))


#takes from 1 to the end, adds on from the start to 1
def RotWord(word):
	return word[1:] + word[:1]


#selects correct value from sbox based on the current word
def SubWord(word):
	sWord = ()

	#loop throug the current word
	for i in range(4):

		#check first char, if its a letter(a-f) get corresponding decimal
		#otherwise just take the value and add 1
		if word[i][0].isdigit() == False:
			row = ord(word[i][0]) - 86
		else:
			row = int(word[i][0])+1

		#repeat above for the seoncd char
		if word[i][1].isdigit() == False:
			col = ord(word[i][1]) - 86
		else:
			col = int(word[i][1])+1

		#get the index base on row and col (16x16 grid)
		sBoxIndex = (row*16) - (17-col)

		#get the value from sbox without prefix
		piece = hex(sbox[sBoxIndex])[2:]

		#check length to ensure leading 0s are not forgotton
		if len(piece) != 2:
			piece = '0' + piece

		#form tuple
		sWord = (*sWord, piece)

	#return string
	return ''.join(sWord)

#used to display the keywords neatly in this form: w0 = 0f 15 71 c9
def prettyPrint(w):
	print("\n\nKeywords: \n")

	for i in range(len(w)):
		print("w" + str(i), "=", w[i][0], w[i][1], w[i][2], w[i][3])


def main():
	#hardcoding input key for demonstration purposes, could be read in from user/program via cmd/gui etc.
	key = ["0f", "15", "71", "c9", "47", "d9", "e8", "59", "0c", "b7", "ad", "d6", "af", "7f", "67", "98"]

	#expand key
	w = keyExpansion(key)
	print(w)

	#display nicely
	print("Key provided: " + "".join(key))
	prettyPrint(w)

if __name__ == '__main__':
	main()

[('0f', '15', '71', 'c9'), ('47', 'd9', 'e8', '59'), ('0c', 'b7', 'ad', 'd6'), ('af', '7f', '67', '98'), ('dc', '90', '37', 'b0'), ('9b', '49', 'df', 'e9'), ('97', 'fe', '72', '3f'), ('38', '81', '15', 'a7'), ('d2', 'c9', '6b', 'b7'), ('49', '80', 'b4', '5e'), ('de', '7e', 'c6', '61'), ('e6', 'ff', 'd3', 'c6'), ('c0', 'af', 'df', '39'), ('89', '2f', '6b', '67'), ('57', '51', 'ad', '06'), ('b1', 'ae', '7e', 'c0'), ('2c', '5c', '65', 'f1'), ('a5', '73', '0e', '96'), ('f2', '22', 'a3', '90'), ('43', '8c', 'dd', '50'), ('58', '9d', '36', 'eb'), ('fd', 'ee', '38', '7d'), ('0f', 'cc', '9b', 'ed'), ('4c', '40', '46', 'bd'), ('71', 'c7', '4c', 'c2'), ('8c', '29', '74', 'bf'), ('83', 'e5', 'ef', '52'), ('cf', 'a5', 'a9', 'ef'), ('37', '14', '93', '48'), ('bb', '3d', 'e7', 'f7'), ('38', 'd8', '08', 'a5'), ('f7', '7d', 'a1', '4a'), ('48', '26', '45', '20'), ('f3', '1b', 'a2', 'd7'), ('cb', 'c3', 'aa', '72'), ('3c', 'be', '0b', '38'), ('fd', '0d', '42', 'cb'), ('0e', '16', 'e0', '1c'), ('c5', 'd5'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

"""
def toBinary(a):
  l,m=[],[]
  for i in a:
    l.append(ord(i))
  for i in l:
    m.append(int(bin(i)[2:]))
  return m
value="Two One Nine Two"
binary_val=toBinary(value)

# Python program to convert binary number
# into hexadecimal number

# Function calculates the decimal equivalent
# to given binary number
def binaryToDecimal(binary):

	binary1 = int(binary)
	decimal, i, n = 0, 0, 0

	while(binary1 != 0):
		dec = binary1 % 10
		decimal = decimal + dec * pow(2, i)
		binary1 = binary1//10
		i += 1
	return(decimal)

# function to convert
# decimal to hexadecimal
def decToHexa(n):

	# char array to store
	# hexadecimal number
	hexaDeciNum = ['0'] * 100

	# counter for hexadecimal
	# number array
	i = 0
	while(n != 0):

		# temporary variable
		# to store remainder
		temp = 0

		# storing remainder
		# in temp variable.
		temp = n % 16

		# check if temp < 10
		if(temp < 10):
			hexaDeciNum[i] = chr(temp + 48)
			i = i + 1
		else:
			hexaDeciNum[i] = chr(temp + 55)
			i = i + 1
		n = int(n / 16)

	# printing hexadecimal number
	# array in reverse order
	j = i - 1
	while(j >= 0):
		print((hexaDeciNum[j]), end="")
		j = j - 1
	print()

# function to convert binary to
# hexadecimal
def binToHexa(n):
	decimal = binaryToDecimal(n)
	print("Hexadecimal equivalent of {}: ".format(n))
	decToHexa(decimal)
for i in range(len(binary_val)):
  binary_val[i]=binToHexa(binary_val[i])
print(binary_val)
"""
def convert_1d_to_2d(l, cols):
    return [l[j:j + cols] for j in range(0, len(l), cols)]
def convertPlain(plain):
		v2=plain.encode('utf-8')
		b=v2.hex()
	#convert from string to hex
	#v2="Two One Nine Two".encode('utf-8')
  #print(b)

  #convert plaintext to 1d array
  #a = '54776f204f6e65204e696e652054776f'
		n = 2
		m1 = [b[i:i+n] for i in range(0, len(b), n)]
   #print ("this is the 1d array"+"",m1)

  #convert 1d array  to 2d array
		my_matrix=convert_1d_to_2d(m1, 4)
  #print("this is the 2d array plaintext"+"",my_matrix)
	# here we need to arrange array to be in rows rather than in columns, so we did some manipulations
  #77
		temp=my_matrix[0][1]
		my_matrix[0][1]=my_matrix[1][0]
		#print(my_matrix)
		#6f
		temp2= my_matrix[0][2]
		my_matrix[0][2]=my_matrix[2][0]
  #print(my_matrix)
  #20
		temp3= my_matrix[0][3]
		my_matrix[0][3]=my_matrix[3][0]
  #4f
		temp4= my_matrix[1][0]
		my_matrix[1][0]=temp
  #print(my_matrix)
  #65
		temp5= my_matrix[1][2]
		my_matrix[1][2]=my_matrix[2][1]
  #print(my_matrix)
  #20
		temp6= my_matrix[1][3]
		my_matrix[1][3]=my_matrix[3][1]
  #print(my_matrix)
  #20
		temp7= my_matrix[1][3]
		my_matrix[2][0]=temp2
  #print(my_matrix)
  #69
		temp8= my_matrix[2][1]
		my_matrix[2][1]=temp5
  #print(my_matrix)
  #65
		temp9= my_matrix[2][3]
		my_matrix[2][3]=my_matrix[3][2]
  #print(my_matrix)
  #3rd row and final one
		my_matrix[3][0]=temp3
		my_matrix[3][1]=temp6
		my_matrix[3][2]=temp9
  #print(my_matrix)
		return my_matrix
#print(convertPlain("One One Nine ome"))
#[['54', '4f', '4e', '20'], ['77', '6e', '69', '54'], ['6f', '65', '6e', '77'], ['20', '20', '65', '6f']]

In [ ]:
#here is the tab of initialize key and make it 2d array
"""
def new_life (a):
  append_str = '0x'
  for i in range(len(a)):
    for j in range(len(a[i])):
      a[i][j]=append_str+a[i][j]
  return a
"""
#our key that wa are working with
key = ["54", "73", "20", "67", "68", "20", "4b", "20", "61", "6d", "75", "46", "74" ,"79", "6e" ,"75"]

#expand key
w= keyExpansion(key)
table= list(map(list, w))
print(table)
"""
key_0x=new_life(table)
my_matrix0x=new_life(my_matrix)
print(key_0x)
print(my_matrix0x)



#test binaryfor our key
bin1 = hex2binary("20")
bin2 = hex2binary("6e")
xord= int(bin1, 2) ^ int(bin2, 2)
hexed = hex(xord)[2:]
if len(hexed) != 8:
		hexed = '0' + hexed
print(hexed)
"""


[['54', '73', '20', '67'], ['68', '20', '4b', '20'], ['61', '6d', '75', '46'], ['74', '79', '6e', '75'], ['e3', 'ec', 'bd', 'f5'], ['8b', 'cc', 'f6', 'd5'], ['ea', 'a1', '83', '93'], ['9e', 'd8', 'ed', 'e6'], ['80', 'b9', '33', 'fe'], ['0b', '75', 'c5', '2b'], ['e1', 'd4', '46', 'b8'], ['7f', '0c', 'ab', '5e'], ['7a', 'db', '6b', '2c'], ['71', 'ae', 'ae', '07'], ['90', '7a', 'e8', 'bf'], ['ef', '76', '43', 'e1'], ['4a', 'c1', '93', 'f3'], ['3b', '6f', '3d', 'f4'], ['ab', '15', 'd5', '4b'], ['44', '63', '96', 'aa'], ['a1', '51', '3f', 'e8'], ['9a', '3e', '02', '1c'], ['31', '2b', 'd7', '57'], ['75', '48', '41', 'fd'], ['d3', 'd2', '6b', '75'], ['49', 'ec', '69', '69'], ['78', 'c7', 'be', '3e'], ['0d', '8f', 'ff', 'c3'], ['e0', 'c4', '45', 'a2'], ['a9', '28', '2c', 'cb'], ['d1', 'ef', '92', 'f5'], ['dc', '60', '6d', '36'], ['b0', 'f8', '40', '24'], ['19', 'd0', '6c', 'ef'], ['c8', '3f', 'fe', '1a'], ['14', '5f', '93', '2c'], ['64', '24', '31', 'de'], ['7d', 'f4', '5d', '31'], ['b5', 'cb'

'\nkey_0x=new_life(table)\nmy_matrix0x=new_life(my_matrix)\nprint(key_0x)\nprint(my_matrix0x)\n\n\n\n#test binaryfor our key\nbin1 = hex2binary("20")\nbin2 = hex2binary("6e")\nxord= int(bin1, 2) ^ int(bin2, 2)\nhexed = hex(xord)[2:]\nif len(hexed) != 8:\n\t\thexed = \'0\' + hexed\nprint(hexed)\n'

In [ ]:
def AddRoundKey(state, roundKey):
    rows, cols = (4, 4)
    arr1 = [[0 for i in range(cols)] for j in range(rows)]
    for i in range(len(state)):
      for j in range(len(state[i])):
        #arr1[i][j] = hex(int(state[i][j], 16) ^ int(roundKey[i][j], 16))
        arr1[i][j]=hexor(state[i][j],roundKey[i][j])
        s=arr1[i][j]
        if len(s)>2:
          arr1[i][j]=s[1:]
    return arr1

"""
rows, cols = (4, 4)
arr = [[0 for i in range(cols)] for j in range(rows)]

for i in range(4):
  for j in range(4):
    arr[i][j]=table[i][j]
print(arr)
print(my_matrix)
m_afterR=AddRoundKey(my_matrix,arr)
print("this output of add round" +"",m_afterR)
"""

"""
bin1 = hex2binary("20")
bin2 = hex2binary("6e")
xord= int(bin1, 2) ^ int(bin2, 2)
hexed = hex(xord)[2:]
if len(hexed) != 8:
		hexed = '0' + hexed
print(hexed)

"""
statem=[['54', '4f', '4e', '20'], ['77', '6e', '69', '54'], ['6f', '65', '6e', '77'], ['20', '20', '65', '6f']]
keym=['54', '73', '20', '67'], ['68', '20', '4b', '20'], ['61', '6d', '75', '46'], ['74', '79', '6e', '75']
print(AddRoundKey(statem,keym))
#['54', '73', '20', '67'], ['68', '20', '4b', '20'], ['61', '6d', '75', '46'], ['74', '79', '6e', '75']
statec=[['4f', '4f', '4e', '20'], ['6e', '6e', '69', '6f'], ['65', '65', '6e', '6d'], ['20', '20', '65', '65']]
print(AddRoundKey(statec,keym))

[['00', '3c', '6e', '47'], ['1f', '4e', '22', '74'], ['0e', '08', '1b', '31'], ['54', '59', '0b', '1a']]
[['1b', '3c', '6e', '47'], ['06', '4e', '22', '4f'], ['04', '08', '1b', '2b'], ['54', '59', '0b', '10']]


In [ ]:
def subytes(temp1):
  temp3=""
  for i in range(4):
    temp=SubWord(temp1[i])
    temp3 = temp3 + temp
  x = 2
  m2 = [temp3[i:i+x] for i in range(0, len(temp3), x)]
  #print(temp3)
  #print(m2)
  result_matrix=[m2[j:j + 4] for j in range(0, len(m2), 4)]
  return result_matrix

"""
mtrx=[['00', '3c', '6e', '47'], ['1f', '4e', '22', '74'], ['0e', '08', '1b', '31'], ['54', '59', '0b', '1a']]
test2= subytes(mtrx)
print(test2)
#print(subytes(m_afterR))

"""
state_s=[['1b', '3c', '6e', '47'], ['06', '4e', '22', '4f'], ['04', '08', '1b', '2b'], ['54', '59', '0b', '10']]
state_s1=[['00', '3c', '6e', '47'], ['1f', '4e', '22', '74'], ['0e', '08', '1b', '31'], ['54', '59', '0b', '1a']]
print(subytes(state_s))
print(subytes(state_s1))



[['af', 'eb', '9f', 'a0'], ['6f', '2f', '93', '84'], ['f2', '30', 'af', 'f1'], ['20', 'cb', '2b', 'ca']]
[['63', 'eb', '9f', 'a0'], ['c0', '2f', '93', '92'], ['ab', '30', 'af', 'c7'], ['20', 'cb', '2b', 'a2']]


In [ ]:
def shiftrows(state):
  #2 row
  temp= state[1][3]
  state[1][3]=state[1][0]
  state[1][0]=state[1][1]
  state[1][1]=state[1][2]
  state[1][2]=temp
  # 3 row
  temp1=state[2][0]
  temp2=state[2][1]
  state[2][0]=state[2][2]
  state[2][1]=state[2][3]
  state[2][2]=temp1
  state[2][3]=temp2
  #4 row
  temp3=state[3][0]
  state[3][0]=state[3][3]
  state[3][3]=state[3][2]
  state[3][2]=state[3][1]
  state[3][1]=temp3
  return state

state=[['6d', '2b', '93', '20'], ['30', '23', '23', '94'], ['ab', '30', 'af', 'c7'], ['20', 'cb', '2b', 'a2']]
print(shiftrows(state))

[['6d', '2b', '93', '20'], ['23', '23', '94', '30'], ['af', 'c7', 'ab', '30'], ['a2', '20', 'cb', '2b']]


In [ ]:
"""
MIX_COLUMN_MATRIX = (
    (0x2, 0x3, 0x1, 0x1),
    (0x1, 0x2, 0x3, 0x1),
    (0x1, 0x1, 0x2, 0x3),
    (0x3, 0x1, 0x1, 0x2),
)
RIJNDAEL_LOG_TABLE = (
    0x00, 0x00, 0x19, 0x01, 0x32, 0x02, 0x1a, 0xc6,
    0x4b, 0xc7, 0x1b, 0x68, 0x33, 0xee, 0xdf, 0x03,
    0x64, 0x04, 0xe0, 0x0e, 0x34, 0x8d, 0x81, 0xef,
    0x4c, 0x71, 0x08, 0xc8, 0xf8, 0x69, 0x1c, 0xc1,
    0x7d, 0xc2, 0x1d, 0xb5, 0xf9, 0xb9, 0x27, 0x6a,
    0x4d, 0xe4, 0xa6, 0x72, 0x9a, 0xc9, 0x09, 0x78,
    0x65, 0x2f, 0x8a, 0x05, 0x21, 0x0f, 0xe1, 0x24,
    0x12, 0xf0, 0x82, 0x45, 0x35, 0x93, 0xda, 0x8e,
    0x96, 0x8f, 0xdb, 0xbd, 0x36, 0xd0, 0xce, 0x94,
    0x13, 0x5c, 0xd2, 0xf1, 0x40, 0x46, 0x83, 0x38,
    0x66, 0xdd, 0xfd, 0x30, 0xbf, 0x06, 0x8b, 0x62,
    0xb3, 0x25, 0xe2, 0x98, 0x22, 0x88, 0x91, 0x10,
    0x7e, 0x6e, 0x48, 0xc3, 0xa3, 0xb6, 0x1e, 0x42,
    0x3a, 0x6b, 0x28, 0x54, 0xfa, 0x85, 0x3d, 0xba,
    0x2b, 0x79, 0x0a, 0x15, 0x9b, 0x9f, 0x5e, 0xca,
    0x4e, 0xd4, 0xac, 0xe5, 0xf3, 0x73, 0xa7, 0x57,
    0xaf, 0x58, 0xa8, 0x50, 0xf4, 0xea, 0xd6, 0x74,
    0x4f, 0xae, 0xe9, 0xd5, 0xe7, 0xe6, 0xad, 0xe8,
    0x2c, 0xd7, 0x75, 0x7a, 0xeb, 0x16, 0x0b, 0xf5,
    0x59, 0xcb, 0x5f, 0xb0, 0x9c, 0xa9, 0x51, 0xa0,
    0x7f, 0x0c, 0xf6, 0x6f, 0x17, 0xc4, 0x49, 0xec,
    0xd8, 0x43, 0x1f, 0x2d, 0xa4, 0x76, 0x7b, 0xb7,
    0xcc, 0xbb, 0x3e, 0x5a, 0xfb, 0x60, 0xb1, 0x86,
    0x3b, 0x52, 0xa1, 0x6c, 0xaa, 0x55, 0x29, 0x9d,
    0x97, 0xb2, 0x87, 0x90, 0x61, 0xbe, 0xdc, 0xfc,
    0xbc, 0x95, 0xcf, 0xcd, 0x37, 0x3f, 0x5b, 0xd1,
    0x53, 0x39, 0x84, 0x3c, 0x41, 0xa2, 0x6d, 0x47,
    0x14, 0x2a, 0x9e, 0x5d, 0x56, 0xf2, 0xd3, 0xab,
    0x44, 0x11, 0x92, 0xd9, 0x23, 0x20, 0x2e, 0x89,
    0xb4, 0x7c, 0xb8, 0x26, 0x77, 0x99, 0xe3, 0xa5,
    0x67, 0x4a, 0xed, 0xde, 0xc5, 0x31, 0xfe, 0x18,
    0x0d, 0x63, 0x8c, 0x80, 0xc0, 0xf7, 0x70, 0x07,
)
RIJNDAEL_EXP_TABLE = (
    0x01, 0x03, 0x05, 0x0F, 0x11, 0x33, 0x55, 0xFF,
    0x1A, 0x2E, 0x72, 0x96, 0xA1, 0xF8, 0x13, 0x35,
    0x5F, 0xE1, 0x38, 0x48, 0xD8, 0x73, 0x95, 0xA4,
    0xF7, 0x02, 0x06, 0x0A, 0x1E, 0x22, 0x66, 0xAA,
    0xE5, 0x34, 0x5C, 0xE4, 0x37, 0x59, 0xEB, 0x26,
    0x6A, 0xBE, 0xD9, 0x70, 0x90, 0xAB, 0xE6, 0x31,
    0x53, 0xF5, 0x04, 0x0C, 0x14, 0x3C, 0x44, 0xCC,
    0x4F, 0xD1, 0x68, 0xB8, 0xD3, 0x6E, 0xB2, 0xCD,
    0x4C, 0xD4, 0x67, 0xA9, 0xE0, 0x3B, 0x4D, 0xD7,
    0x62, 0xA6, 0xF1, 0x08, 0x18, 0x28, 0x78, 0x88,
    0x83, 0x9E, 0xB9, 0xD0, 0x6B, 0xBD, 0xDC, 0x7F,
    0x81, 0x98, 0xB3, 0xCE, 0x49, 0xDB, 0x76, 0x9A,
    0xB5, 0xC4, 0x57, 0xF9, 0x10, 0x30, 0x50, 0xF0,
    0x0B, 0x1D, 0x27, 0x69, 0xBB, 0xD6, 0x61, 0xA3,
    0xFE, 0x19, 0x2B, 0x7D, 0x87, 0x92, 0xAD, 0xEC,
    0x2F, 0x71, 0x93, 0xAE, 0xE9, 0x20, 0x60, 0xA0,
    0xFB, 0x16, 0x3A, 0x4E, 0xD2, 0x6D, 0xB7, 0xC2,
    0x5D, 0xE7, 0x32, 0x56, 0xFA, 0x15, 0x3F, 0x41,
    0xC3, 0x5E, 0xE2, 0x3D, 0x47, 0xC9, 0x40, 0xC0,
    0x5B, 0xED, 0x2C, 0x74, 0x9C, 0xBF, 0xDA, 0x75,
    0x9F, 0xBA, 0xD5, 0x64, 0xAC, 0xEF, 0x2A, 0x7E,
    0x82, 0x9D, 0xBC, 0xDF, 0x7A, 0x8E, 0x89, 0x80,
    0x9B, 0xB6, 0xC1, 0x58, 0xE8, 0x23, 0x65, 0xAF,
    0xEA, 0x25, 0x6F, 0xB1, 0xC8, 0x43, 0xC5, 0x54,
    0xFC, 0x1F, 0x21, 0x63, 0xA5, 0xF4, 0x07, 0x09,
    0x1B, 0x2D, 0x77, 0x99, 0xB0, 0xCB, 0x46, 0xCA,
    0x45, 0xCF, 0x4A, 0xDE, 0x79, 0x8B, 0x86, 0x91,
    0xA8, 0xE3, 0x3E, 0x42, 0xC6, 0x51, 0xF3, 0x0E,
    0x12, 0x36, 0x5A, 0xEE, 0x29, 0x7B, 0x8D, 0x8C,
    0x8F, 0x8A, 0x85, 0x94, 0xA7, 0xF2, 0x0D, 0x17,
    0x39, 0x4B, 0xDD, 0x7C, 0x84, 0x97, 0xA2, 0xFD,
    0x1C, 0x24, 0x6C, 0xB4, 0xC7, 0x52, 0xF6, 0x01,
)
def rijndael_mul(a, b):
    if a == 0 or b == 0:
        return 0
    return RIJNDAEL_EXP_TABLE[(RIJNDAEL_LOG_TABLE[a] + RIJNDAEL_LOG_TABLE[b]) % 0xFF]


def mix_column(data, matrix):
    data_mixed = []
    for row in range(3):
        mixed = 0
        for column in range(3):
            # xor is (+) and (-)
            mixed ^= rijndael_mul(data[column], matrix[row][column])
        data_mixed.append(mixed)
    return data_mixed


def mix_columns(data, matrix=MIX_COLUMN_MATRIX):
    data_mixed = []
    for i in range(4):
        column = data[i * 4: (i + 1) * 4]
        data_mixed += mix_column(column, matrix)
    return data_mixed
state_bin1d=[1100011, 11101011, 10011111, 10100000]
#, 101111, 10010011, 10010010, 11000000,10101111, 11000111, 10101011, 110000,10100010, 100000, 11001011, 101011]
print(mix_columns(state_bin1d))
"""

'\nMIX_COLUMN_MATRIX = (\n    (0x2, 0x3, 0x1, 0x1),\n    (0x1, 0x2, 0x3, 0x1),\n    (0x1, 0x1, 0x2, 0x3),\n    (0x3, 0x1, 0x1, 0x2),\n)\nRIJNDAEL_LOG_TABLE = (\n    0x00, 0x00, 0x19, 0x01, 0x32, 0x02, 0x1a, 0xc6,\n    0x4b, 0xc7, 0x1b, 0x68, 0x33, 0xee, 0xdf, 0x03,\n    0x64, 0x04, 0xe0, 0x0e, 0x34, 0x8d, 0x81, 0xef,\n    0x4c, 0x71, 0x08, 0xc8, 0xf8, 0x69, 0x1c, 0xc1,\n    0x7d, 0xc2, 0x1d, 0xb5, 0xf9, 0xb9, 0x27, 0x6a,\n    0x4d, 0xe4, 0xa6, 0x72, 0x9a, 0xc9, 0x09, 0x78,\n    0x65, 0x2f, 0x8a, 0x05, 0x21, 0x0f, 0xe1, 0x24,\n    0x12, 0xf0, 0x82, 0x45, 0x35, 0x93, 0xda, 0x8e,\n    0x96, 0x8f, 0xdb, 0xbd, 0x36, 0xd0, 0xce, 0x94,\n    0x13, 0x5c, 0xd2, 0xf1, 0x40, 0x46, 0x83, 0x38,\n    0x66, 0xdd, 0xfd, 0x30, 0xbf, 0x06, 0x8b, 0x62,\n    0xb3, 0x25, 0xe2, 0x98, 0x22, 0x88, 0x91, 0x10,\n    0x7e, 0x6e, 0x48, 0xc3, 0xa3, 0xb6, 0x1e, 0x42,\n    0x3a, 0x6b, 0x28, 0x54, 0xfa, 0x85, 0x3d, 0xba,\n    0x2b, 0x79, 0x0a, 0x15, 0x9b, 0x9f, 0x5e, 0xca,\n    0x4e, 0xd4, 0xac, 0xe5, 0xf3, 0x73, 0xa7

In [ ]:
def flatten_list(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

import binascii, copy, random, sys
# Galois Multiplication
def galoisMult(a, b):
    p = 0
    hiBitSet = 0
    for i in range(8):
        if b & 1 == 1:
            p ^= a
        hiBitSet = a & 0x80
        a <<= 1
        if hiBitSet == 0x80:
            a ^= 0x1b
        b >>= 1
    return p % 256

#mixColumn takes a column and does stuff
def mixColumn(column):
    temp = copy.copy(column)
    column[0] = galoisMult(temp[0],2) ^ galoisMult(temp[3],1) ^ \
                galoisMult(temp[2],1) ^ galoisMult(temp[1],3)
    column[1] = galoisMult(temp[1],2) ^ galoisMult(temp[0],1) ^ \
                galoisMult(temp[3],1) ^ galoisMult(temp[2],3)
    column[2] = galoisMult(temp[2],2) ^ galoisMult(temp[1],1) ^ \
                galoisMult(temp[0],1) ^ galoisMult(temp[3],3)
    column[3] = galoisMult(temp[3],2) ^ galoisMult(temp[2],1) ^ \
                galoisMult(temp[1],1) ^ galoisMult(temp[0],3)
# mixColumns is a wrapper for mixColumn - generates a "virtual" column from
# the state table and applies the weird galois math
def mixColumns(state):
    for i in range (len(state)):
       for j in range(4):
          state[i][j] =int(bin(int(state[i][j],16))[2:])
    state=flatten_list(state)
    for i in range(4):
        column = []
        # create the column by taking the same item out of each "virtual" row
        for j in range(4):
            column.append(state[j*4+i])

        # apply mixColumn on our virtual column
        mixColumn(column)

        # transfer the new values back into the state table
        for j in range(4):
            state[j*4+i] = column[j]
    return state
def mixColumns_final(state):
  test=mixColumns(state)
  for i in range (len(test)):
    test[i]=hex(test[i])
  test1= convert_1d_to_2d(test,4)
  return test1

state50=[['33', 'eb', '5f', '30'], ['2f', '93', '92', 'c0'], ['af', 'c7', 'ab', '30'], ['a2', '20', 'cb', '2b']]
#state_bin1d=[1100011, 11101011, 10011111, 10100000, 101111, 10010011, 10010010, 11000000,10101111, 11000111, 10101011, 110000,10100010, 100000, 11001011, 101011]
#state3=[0x63, 0xeb, 0x9f, 0xa0, 0x2f, 0x93, 0x92, 0xc0,0xaf, 0xc7, 0xab, 0x30,0xa2, 0x20, 0xcb, 0x2b]
#for i in range (16):
#   state_bin1d[i]=int(state_bin1d[i])

#print(mixColumns(state3))
#[['0x92', '0x9f', '0xc0', '0x38'], ['0xad', '0xaf', '0x4e', '0xe3'], ['0x8c', '0x6d', '0x95', '0x35'], ['0xf2', '0x1a', '0xc6', '0x2d']]
print(mixColumns_final(state50))

[['0x32', '0x9f', '0x40', '0x3'], ['0xfd', '0xaf', '0xe', '0x73'], ['0xdc', '0x6d', '0xd5', '0xa5'], ['0x2', '0x1a', '0x6', '0x86']]


In [ ]:
"""
i=0
k=0
while k <11:
  rows,cols = (4,4)
  arr = [[0 for i in range(cols)] for j in range(rows)]
  while i < k+4:
    for j in range (4):
       arr[i][j]=table[i][j]
    i=i+1
  k=k+4
  i=k
  print(arr)
"""

' \ni=0\nk=0\nwhile k <11:\n  rows,cols = (4,4)\n  arr = [[0 for i in range(cols)] for j in range(rows)]\n  while i < k+4:\n    for j in range (4):\n       arr[i][j]=table[i][j]\n    i=i+1\n  k=k+4\n  i=k\n  print(arr)\n'

In [ ]:
rkeys = table
print(rkeys)
#Round Keys generate
R0 = []

R0.append(rkeys[0])
R0.append(rkeys[1])
R0.append(rkeys[2])
R0.append(rkeys[3])
R1 = []
R1.append(rkeys[4])
R1.append(rkeys[5])
R1.append(rkeys[6])
R1.append(rkeys[7])
R2 = []
R2.append(rkeys[8])
R2.append(rkeys[9])
R2.append(rkeys[10])
R2.append(rkeys[11])
R3 = []
R3.append(rkeys[12])
R3.append(rkeys[13])
R3.append(rkeys[14])
R3.append(rkeys[15])
R4 = []
R4.append(rkeys[16])
R4.append(rkeys[17])
R4.append(rkeys[18])
R4.append(rkeys[19])
R5 = []
R5.append(rkeys[20])
R5.append(rkeys[21])
R5.append(rkeys[22])
R5.append(rkeys[23])
R6 = []
R6.append(rkeys[24])
R6.append(rkeys[25])
R6.append(rkeys[26])
R6.append(rkeys[27])
R7 = []
R7.append(rkeys[28])
R7.append(rkeys[29])
R7.append(rkeys[30])
R7.append(rkeys[31])
R8 = []
R8.append(rkeys[32])
R8.append(rkeys[33])
R8.append(rkeys[34])
R8.append(rkeys[35])
R9 = []
R9.append(rkeys[36])
R9.append(rkeys[37])
R9.append(rkeys[38])
R9.append(rkeys[39])
R10 = []
R10.append(rkeys[40])
R10.append(rkeys[41])
R10.append(rkeys[42])
R10.append(rkeys[43])
print(R0)
print(R1)


[['54', '73', '20', '67'], ['68', '20', '4b', '20'], ['61', '6d', '75', '46'], ['74', '79', '6e', '75'], ['e3', 'ec', 'bd', 'f5'], ['8b', 'cc', 'f6', 'd5'], ['ea', 'a1', '83', '93'], ['9e', 'd8', 'ed', 'e6'], ['80', 'b9', '33', 'fe'], ['0b', '75', 'c5', '2b'], ['e1', 'd4', '46', 'b8'], ['7f', '0c', 'ab', '5e'], ['7a', 'db', '6b', '2c'], ['71', 'ae', 'ae', '07'], ['90', '7a', 'e8', 'bf'], ['ef', '76', '43', 'e1'], ['4a', 'c1', '93', 'f3'], ['3b', '6f', '3d', 'f4'], ['ab', '15', 'd5', '4b'], ['44', '63', '96', 'aa'], ['a1', '51', '3f', 'e8'], ['9a', '3e', '02', '1c'], ['31', '2b', 'd7', '57'], ['75', '48', '41', 'fd'], ['d3', 'd2', '6b', '75'], ['49', 'ec', '69', '69'], ['78', 'c7', 'be', '3e'], ['0d', '8f', 'ff', 'c3'], ['e0', 'c4', '45', 'a2'], ['a9', '28', '2c', 'cb'], ['d1', 'ef', '92', 'f5'], ['dc', '60', '6d', '36'], ['b0', 'f8', '40', '24'], ['19', 'd0', '6c', 'ef'], ['c8', '3f', 'fe', '1a'], ['14', '5f', '93', '2c'], ['64', '24', '31', 'de'], ['7d', 'f4', '5d', '31'], ['b5', 'cb'

In [ ]:
#AES ENCRYPTIO
"""
def encrypt(state):
  states= convertPlain(state)
  states= AddRoundKey(states,R0)
  for i in range(10):

     states= subytes(states)
     print(states)
     states= shiftrows(states)
     #print(states)
     states= mixColumns_final(states)
     #print(states)
     if i==1:
           states= AddRoundKey(states,R1)
     if i==2:
              states= AddRoundKey(states,R2)

     if i==3:
              states=AddRoundKey(states,R3)

     if i==4:
              states=AddRoundKey(states,R4)

     if i==5:
              states=AddRoundKey(states,R5)

     if i==6:
              states=AddRoundKey(states,R6)

     if i==7:
              states=AddRoundKey(states,R7)

     if i==8:

              states=AddRoundKey(states,R8)

     if i==9:
             states=AddRoundKey(states,R9)



  states= subytes(states)
  states= shiftrows(states)
  print(states)
  states=AddRoundKey(states,R10)
  print(states)
  return states


print(encrypt("Two One Nine Two"))
print(encrypt("bcw bbr nibr mmm"))
#[['8d', 'dd', '09', '11'], ['f0', '29', '54', '20'], ['45', 'e2', 'f7', '0b'], ['e4', '76', 'c7', '0c']]
"""

'\ndef encrypt(state):\n  states= convertPlain(state)\n  states= AddRoundKey(states,R0)\n  for i in range(10):\n     \n     states= subytes(states)\n     print(states)\n     states= shiftrows(states)\n     #print(states)\n     states= mixColumns_final(states)\n     #print(states)\n     if i==1:\n           states= AddRoundKey(states,R1)\n     if i==2:\n              states= AddRoundKey(states,R2)\n\n     if i==3:\n              states=AddRoundKey(states,R3)\n\n     if i==4:\n              states=AddRoundKey(states,R4)\n\n     if i==5:\n              states=AddRoundKey(states,R5)\n\n     if i==6:\n              states=AddRoundKey(states,R6)\n\n     if i==7:\n              states=AddRoundKey(states,R7)\n\n     if i==8:\n\n              states=AddRoundKey(states,R8)\n\n     if i==9:\n             states=AddRoundKey(states,R9)\n    \n\n    \n  states= subytes(states)\n  states= shiftrows(states)\n  print(states)\n  states=AddRoundKey(states,R10)\n  print(states)\n  return states\n\n\nprint

In [ ]:
#Encryption
#ptext= convertPlain(plaintext)
def encryption(plaintext):
  ctext = AddRoundKey(plaintext, R0)
  round = 1
  while round < 10:
   ctext = subytes(ctext)
   ctext = shiftrows(ctext)
   ctext = mixColumns_final(ctext)
   if round == 1:
    ctext = AddRoundKey(ctext, R1)
    round += 1
   elif round == 2:
    ctext = AddRoundKey(ctext, R2)
    round += 1
   elif round == 3:
    ctext = AddRoundKey(ctext, R3)
    round += 1
   elif round == 4:
    ctext = AddRoundKey(ctext, R4)
    round += 1
   elif round == 5:
    ctext = AddRoundKey(ctext, R5)
    round += 1
   elif round == 6:
    ctext = AddRoundKey(ctext, R6)
    round += 1
   elif round == 7:
    ctext = AddRoundKey(ctext, R7)
    round += 1
   elif round == 8:
    ctext = AddRoundKey(ctext, R8)
    round += 1
   elif round == 9:
    ctext = AddRoundKey(ctext, R9)
    round += 1
#encryption for round 10
  ctext = subytes(ctext)
  ctext = shiftrows(ctext)
  ctext = AddRoundKey(ctext, R10)
  return ctext
#print(encryption("Two One Nine Two"))
#print(encryption("bcw bbr nibr mmm"))

In [ ]:
user= input("Please enter plaintext to be encrypted: ")

Please enter plaintext to be encrypted: Two one nine two


In [ ]:
pconvert=convertPlain(user)
print("This is the encrptyion of your plaintext :",encryption(pconvert))

This is the encrptyion of your plaintext : [['8b', 'd1', '28', 'e3'], ['18', '95', '4a', '3f'], ['0f', '60', 'c0', 'ef'], ['a8', 'd8', 'd9', 'b2']]


# ***Task Two***

In [ ]:
def pad(text, block_size):

    #Performs padding on the given plaintext to ensure that it is a multiple
    #of the given block_size value in the parameter. Uses the PKCS7 standard
    #for performing padding.

    no_of_blocks = math.ceil(len(text)/float(block_size))
    pad_value = int(no_of_blocks * block_size - len(text))

    if pad_value == 0:
        return text + chr(block_size) * block_size
    else:
        return text + chr(pad_value) * pad_value
pad(user,16)

'Two one nine two\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'

In [ ]:
"""

def pad1(plaintext):

    padding_len = 16 - (len(plaintext) % 16)
    padding = bytes([padding_len] * padding_len)
    arr=[1,2]
    arr[0]=plaintext
    arr[1]=padding
    return arr
print(pad1(user))
v2=b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'.encode('utf-8')
b=v2.hex()
print(b)
"""

"\n\ndef pad1(plaintext):\n  \n    padding_len = 16 - (len(plaintext) % 16)\n    padding = bytes([padding_len] * padding_len)\n    arr=[1,2]\n    arr[0]=plaintext \n    arr[1]=padding\n    return arr\nprint(pad1(user))\nv2=b'\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10'.encode('utf-8')\nb=v2.hex()\nprint(b)\n"

In [ ]:
def split_blocks(user, block_size=16):
        assert len(user) % block_size == 0
        return [user[i:i+16] for i in range(0, len(user), block_size)]
ppad=pad(user,16)
print(split_blocks(ppad,16))

['Two one nine two', '\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10\x10']


In [ ]:
def ecb_encrypt( data):
    ppad=pad(data,16)
    psplit=split_blocks(ppad,16)
    c=[]
    for i in range(len(psplit)):
      psplit[i]=convertPlain(psplit[i])
      c.append(encryption(psplit[i]))

    return c

print(ecb_encrypt(user))
print(ecb_encrypt('Ome tso sige mmm'))

[[['8b', 'd1', '28', 'e3'], ['18', '95', '4a', '3f'], ['0f', '60', 'c0', 'ef'], ['a8', 'd8', 'd9', 'b2']], [['58', 'ca', 'a6', 'c1'], ['ff', '29', '3e', '4f'], ['b9', '7e', '44', '12'], ['7b', '56', 'e0', '97']]]
[[['f4', '85', 'ae', 'cc'], ['c4', '36', 'd2', '98'], ['f6', 'f7', '79', 'c7'], ['1e', '9f', '6f', 'dd']], [['58', 'ca', 'a6', 'c1'], ['ff', '29', '3e', '4f'], ['b9', '7e', '44', '12'], ['7b', '56', 'e0', '97']]]


In [ ]:
import os
#xor blocks
def xorblocks(ptext, iv):
  ciphertext = []
  for i in range(len(ptext)): #n of arrays
   res1=[]
   for j in range(len(ptext[i])): #n elements in array
      x = hexor(ptext[i][j], iv[i][j])
      if(len(x)>2):
        res1.append(x[1:])
      else:
        res1.append(x)
   ciphertext.append(res1)
  return ciphertext

#generate initialization vector
def generate_random_iv(iv_length):
    return bytes.hex(os.urandom(iv_length))

In [ ]:
import textwrap
def cbc_encrypt(data):
    plaintext=pad(data,16)
    psplit=split_blocks(plaintext,16)
    iv=generate_random_iv(16)
    ivarray = textwrap.wrap(iv, 2)
    ivmatrix = np.reshape(ivarray, (4,4))
    previous=ivmatrix
    #p=convertPlain(psplit[0])
    #p2=xorblocks(p, previous)
    #c_1=encryption(p2)
    cx=[]
    #c.append(c_1)
    #c[0]=encryption(psplit[0])
    #print(c[0])
    for i in range(len(psplit)):
    #for i in split_blocks(plaintext):
      pp=convertPlain(psplit[i])
      c=encryption(xorblocks(pp,previous))
      cx.append(c)
      previous=c
    return cx
print(cbc_encrypt('Ome tso sige mmm'))


[[['b4', '5c', 'ca', '38'], ['38', '58', '26', '86'], ['f2', '79', 'bc', '32'], ['62', 'cd', '33', 'bd']], [['11', 'c9', '7e', 'ff'], ['2e', '96', 'a8', '94'], ['fa', '53', 'af', 'e5'], ['0a', '82', '2e', 'b7']]]


GUI for cbc and ebc test

In [ ]:
p_ebc=input("Please enter a text to be encrypted using ebc mode of encryption: ")

Please enter a text to be encrypted using ebc mode of encryption: two one nine two


In [ ]:
print(ecb_encrypt(p_ebc))

[[['72', '4e', 'fd', 'b8'], ['43', 'd2', '5d', 'f5'], ['c7', '1f', 'ff', '99'], ['e6', 'fe', '25', '76']], [['58', 'ca', 'a6', 'c1'], ['ff', '29', '3e', '4f'], ['b9', '7e', '44', '12'], ['7b', '56', 'e0', '97']]]


In [ ]:
p_cbc=input("Please enter a text to be encrypted using cbc mode of encryption: ")

Please enter a text to be encrypted using cbc mode of encryption: two one nine two


In [ ]:
print(cbc_encrypt(p_cbc))

[[['e3', '5b', '5d', 'b7'], ['35', '6a', '31', '9a'], ['4f', '81', '63', '38'], ['12', '08', '4d', 'c9']], [['d7', '6e', 'e9', '0a'], ['f8', '27', '69', '22'], ['b1', '71', '71', '82'], ['b3', '5d', '13', 'bf']]]
